1. Loading the dataset

1.1. Launch commands to automatically reload modules

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1.2. Import the pandas and numpy package

In [4]:
import pandas as pd
import numpy as np

1.3. Load the training and test dataset into a dataframes called df_train and df_test

In [5]:
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')

C:\Users\paiyn\AppData\Local\Temp\ipykernel_10728\1557408954.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../data/raw/train.csv')
